In [14]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (14, 11)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [15]:
df = pd.read_csv('train.csv')

In [16]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [18]:
# Заполним пропуски и перекодируем категориальные переменные

In [19]:
for x in df.columns:
    if df[x].dtype == 'object':
        df[x] = df[x].fillna('NA')
    elif df[x].dtype == 'int64':
        df[x] = df[x].fillna(0)
    elif df[x].dtype == 'float64':
        df[x] = df[x].fillna(0)

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
le = LabelEncoder()

In [22]:
le.fit_transform(np.array(df['SaleType'].values))

array([8, 8, 8, ..., 8, 8, 8], dtype=int64)

In [28]:
col = [] 
col_scale = []
for x in df.columns:
    if df[x].dtype == 'object':
        col.append(x)
    if df[x].dtype != 'object' and x != 'SalePrice':
        col_scale.append(x)
        df[x] = le.fit_transform(df[x])

In [29]:
y = df.SalePrice
X = df.drop('SalePrice', axis=1)

In [30]:
col = X.columns

In [31]:
from sklearn.preprocessing import StandardScaler

In [27]:
scaler = StandardScaler()
scaler.fit(X)
X_scale = pd.DataFrame(scaler.transform(X[col_scale]), columns = col_scale)

ValueError: could not convert string to float: 'Normal'

In [32]:
X = pd.concat([X[col], X_scale], axis = 1)

NameError: name 'X_scale' is not defined

In [33]:
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0,5,RL,37,327,Pave,NA,Reg,Lvl,AllPub,...,0,0,NA,NA,NA,0,1,2,WD,Normal
1,1,0,RL,52,498,Pave,NA,Reg,Lvl,AllPub,...,0,0,NA,NA,NA,0,4,1,WD,Normal
2,2,5,RL,40,702,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,NA,0,8,2,WD,Normal
3,3,6,RL,32,489,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,NA,0,1,0,WD,Abnorml
4,4,5,RL,56,925,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,NA,0,11,2,WD,Normal


In [ ]:
num_folds = 10
seed = 7

In [ ]:
kfold = KFold(n_splits=num_folds, random_state=seed)

In [ ]:
rfr = RandomForestRegressor()
results = cross_val_score(rfr, X, y, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0
))

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.1, random_state=7)

In [ ]:
rfr.fit(X_train, y_train)

In [ ]:
predict = rfr.predict(X_valid)

In [ ]:
metrics.r2_score(y_valid, predict)

In [ ]:
metrics.explained_variance_score(y_valid, predict)

In [ ]:
# Выведем важность признаков

In [ ]:
i = 0
for x in X.columns:
    print(x, '', '{:2f}'.format(rfr.feature_importances_[i]))
    i+=1

# реализуем Блендинг моделей
<img src="img/stacking.png" style="max-width: 680px; display: inline" />

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

In [ ]:
X_train_st, X_test, y_train_st, y_test = train_test_split(X_train, y_train, test_size=.2, random_state=7)

In [ ]:
# выбираем модели
knn3 = KNeighborsRegressor(n_neighbors=3)
knn10 = KNeighborsRegressor(n_neighbors=10)
rg1 = Ridge(alpha=1.1)
las = Lasso()
rfr = RandomForestRegressor()
metamodel = Ridge()

In [ ]:
def model_fit_predict(model, X_train, y_train, X_test, y_test):
    model.fit(X_train,y_train)
    prediction = model.predict(X_test)
    r2_sc = metrics.r2_score(y_test, prediction)
    return prediction, r2_sc

In [ ]:
list_of_models = [knn3, knn10, rg1, las, rfr]

In [ ]:
pred = []
r2_sc = []
for mod in list_of_models:
    a,b = model_fit_predict(mod, X_train_st, y_train_st, X_test, y_test)
    pred.append(a)
    r2_sc.append(b)

In [ ]:
r2_sc

In [ ]:
X_pred = pd.DataFrame(np.array(pred).T)

In [ ]:
X_pred.shape

In [ ]:
metamodel.fit(X_pred, y_test)

In [ ]:
pred = []
r2_sc = []
for mod in list_of_models:
    a,b = model_fit_predict(mod, X_train, y_train, X_valid, y_valid)
    pred.append(a)
    r2_sc.append(b)

In [ ]:
# метрика r2 для различных моделей

In [ ]:
r2_sc

In [ ]:
X_pred = pd.DataFrame(np.array(pred).T)

In [ ]:
y_pred = metamodel.predict(X_pred)

In [ ]:
# метрика r2 для ансамбля моделей

In [ ]:
metrics.r2_score(y_valid, y_pred)

In [ ]:
## Стекинг

Используем идею K-Fold проверки. Выборку разбиваем на фолды, затем последовательно перебираем фолды и обучаем базовые алгоритмы на всех фолдах, кроме одного, а на оставшемся получаем ответы базовых алгоритмов и трактуем их как значения соответствующих признаков на этом фолде. Для получения метапризнаков объектов тестовой выборки базовые алгоритмы обучают на всей обучающей выборке и берут их ответы на тестовой.

<img src="img/stacking-2b.png" style="max-width: 680px; display: inline" />

In [ ]:
def get_meta_features(model, X_train, y_train, X_test, stack_cv):
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(y_test, dtype=float)
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        
        model.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
        meta_train[test_ind] = model.predict(X_train.iloc[test_ind])
        meta_test += model.predict(X_test)
        
    r2_sc1 = metrics.r2_score(y_test, meta_test / stack_cv.n_splits)
    
    return meta_train, meta_test / stack_cv.n_splits, r2_sc1

In [ ]:
from sklearn.model_selection import StratifiedKFold

stack_cv = StratifiedKFold(n_splits=10, random_state=555)

In [ ]:
stack_cv.split(X_train, y_train)

In [ ]:
meta_train = []
meta_test = []
col_names = []
r2_sc_stack = []

In [ ]:
for mod in list_of_models:
    meta_tr, meta_te, r2_score = get_meta_features(mod, X_train, y_train, X_test, stack_cv)
    meta_train.append(meta_tr)
    meta_test.append(meta_te)
    r2_sc_stack.append(r2_score)
    col_names.append(str(mod)+'_pred')

In [ ]:
X_meta_train = pd.DataFrame(np.stack(meta_train, axis=1), columns=col_names)
X_meta_test = pd.DataFrame(np.stack(meta_test, axis=1), columns=col_names)

In [ ]:
#Выведем качество моделей 
r2_sc_stack

In [ ]:
X_meta_train.shape

In [ ]:
X_meta_test.shape

In [ ]:
metamodel.fit(X_meta_train, y_train)

In [ ]:
y_pred_meta_valid = metamodel.predict(X_meta_test)

In [ ]:
# Финальная метрика качества метамодели
metrics.r2_score(y_test, y_pred_meta_valid)